In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
(X, Y), (X1, Y1) = tf.keras.datasets.mnist.load_data()
tf.compat.v1.disable_eager_execution()
print(type(X))
print(X.shape)
print(X.dtype)
print(type(Y))
print(Y.shape)
print(Y.dtype)

<class 'numpy.ndarray'>
(60000, 28, 28)
uint8
<class 'numpy.ndarray'>
(60000,)
uint8


In [3]:
#FOR TRAINING
X = X.reshape(len(X),784)
lk = []
def onehot(xi):
  l = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  l[xi] = 1.0
  return np.array(l)
for i in range(len(Y)):
  lk.append(onehot(Y[i]))
Y = np.array(lk, dtype = float)
X = np.array(X, dtype = float)
print(Y.shape)
print(X.dtype)
print(X.shape)
print(Y.dtype)

(60000, 10)
float64
(60000, 784)
float64


In [4]:
#FOR TESTING
X1 = X1.reshape(len(X1),784)
lk1 = []
def onehot(xi):
  l = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  l[xi] = 1.0
  return np.array(l)
for i in range(len(Y1)):
  lk1.append(onehot(Y1[i]))
Y1 = np.array(lk1, dtype = float)
X1 = np.array(X1, dtype = float)
print(Y1.shape)
print(X1.dtype)
print(X1.shape)
print(Y1.dtype)

(10000, 10)
float64
(10000, 784)
float64


In [0]:
#WEIGHTS INITIALIZER
def init_weights(shape):
  init_random_dist = tf.compat.v1.truncated_normal(shape,stddev=0.1)
  return tf.Variable(init_random_dist)

In [0]:
#BIAS INITIALIZER
def init_bias(shape):
  init_bias_vals = tf.constant(0.1,shape=shape)
  return tf.Variable(init_bias_vals)

In [0]:
#CONVOLUTION
def conv(x,W):
  #x (input) -->[batch, height, width, channels]
  #W (weights) --> [filter H, filter W, channels in, channels out]
  return tf.nn.conv2d(x,W,strides = [1,1,1,1], padding = 'SAME')

In [0]:
#POOLING LAYER
def pool(x):
  #x --> same as above
  return tf.nn.max_pool(x,ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
#CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
  W = init_weights(shape)
  b = init_bias([shape[3]])
  return tf.nn.relu(conv(input_x, W)+b)

In [0]:
#NORMAL (FULLY CONNECTED)
def normal_full_layer(input_layer,size):
  input_size = int(input_layer.get_shape()[1])
  W = init_weights([input_size,size])
  b = init_bias([size])
  return tf.matmul(input_layer,W)+b

In [0]:
#PLACEHOLDERS
x = tf.compat.v1.placeholder(tf.float32, shape=[None, 784])
y_true = tf.compat.v1.placeholder(tf.float32, shape=[None,10])

In [12]:
#LAYERS
x_image = tf.reshape(x, [-1,28,28,1])
convo1 = convolutional_layer(x_image, shape=[5,5,1,32])
#shape =[a,b,c,d]
#here a,b = batch size; c= input_featres , d=output_features
convo1_pooling  = pool(convo1)
convo2 = convolutional_layer(convo1_pooling, shape=[5,5,32,64])
convo2_pooling = pool(convo2)

convo2_flat = tf.reshape(convo2_pooling, [-1,7*7*64])
full_layer_1 = tf.nn.relu(normal_full_layer(convo2_flat ,1024))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#DROPOUT
drop_prob = tf.compat.v1.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_1, rate = drop_prob)

In [0]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [0]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [0]:
#OPTIMIZER
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [0]:
init = tf.compat.v1.global_variables_initializer()

In [18]:
#CREATE SESSION
counter = 0 
steps = 25000
batch_size = 50
drop_probability = 0.3  
init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess:
  sess.run(init)
  for i in range(steps):
    if(counter==len(X)):
      counter = 0
    batch_x  , batch_y = X[counter:counter+batch_size , :] , Y[counter:counter+batch_size , :]
    counter+= batch_size
    sess.run(train, feed_dict={x:batch_x, y_true:batch_y, drop_prob:drop_probability})

    if (i+1)%1000==0:
      print('ON STEP : {}'.format(i+1))
      matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
      acc = tf.reduce_mean(tf.cast(matches,tf.float32))
      print("Accuracy : ", sess.run(acc,feed_dict={ x:X, y_true:Y, drop_prob:0.0})) 
      print()
  print()
  print("Final Accuracy on test set")
  print("Accuracy : ", sess.run(acc,feed_dict={ x:X1, y_true:Y1, drop_prob:0.0})) 

  #SAVING MODEL
  saver = tf.compat.v1.train.Saver()
  saver.save(sess, 'convolution_result/mnist_convo.ckpt')

ON STEP : 1000
Accuracy :  0.9615833

ON STEP : 2000
Accuracy :  0.97838336

ON STEP : 3000
Accuracy :  0.9780833

ON STEP : 4000
Accuracy :  0.9819833

ON STEP : 5000
Accuracy :  0.98175

ON STEP : 6000
Accuracy :  0.9891833

ON STEP : 7000
Accuracy :  0.9835

ON STEP : 8000
Accuracy :  0.98361665

ON STEP : 9000
Accuracy :  0.9862667

ON STEP : 10000
Accuracy :  0.98641664

ON STEP : 11000
Accuracy :  0.9887667

ON STEP : 12000
Accuracy :  0.98721665

ON STEP : 13000
Accuracy :  0.99185

ON STEP : 14000
Accuracy :  0.9916

ON STEP : 15000
Accuracy :  0.99345

ON STEP : 16000
Accuracy :  0.98641664

ON STEP : 17000
Accuracy :  0.99081665

ON STEP : 18000
Accuracy :  0.9925

ON STEP : 19000
Accuracy :  0.9952

ON STEP : 20000
Accuracy :  0.9944

ON STEP : 21000
Accuracy :  0.9936

ON STEP : 22000
Accuracy :  0.99675

ON STEP : 23000
Accuracy :  0.9906

ON STEP : 24000
Accuracy :  0.9967333

ON STEP : 25000
Accuracy :  0.9957167


Final Accuracy on test set
Accuracy :  0.9865
